In [ ]:
import cv2
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
# Load your trained model
model = load_model('saved_model.h5')

# Initialize the camera
cap = cv2.VideoCapture(0)

# Load pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    frame = cv2.flip(frame, 1)
    
    # Convert to RGB (optional for face detection)
    RGB_IMG = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = face_cascade.detectMultiScale(RGB_IMG, 1.3, 5)

    for (x, y, w, h) in faces:
        # Crop the face with a margin
        margin = 30
        x1, y1, x2, y2 = x-margin, y-margin, x+w+margin, y+h+margin
        face = frame[y1:y2, x1:x2]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        # Resize to the model's input size
        face_resized = cv2.resize(face, (178, 218))

        # Preprocess the image
        face_resized = np.expand_dims(face_resized, axis=0)
        face_resized = face_resized / 255.0  # Normalize pixel values
        
        # Predict
        prediction = model.predict(face_resized)

        # Determine the label
        if prediction[0][0] > 0.5:
            label = 'Smile'
            color = (255, 0, 0)
        else:
            label = 'Not Smile'
            color = (0, 0, 255)

        # Draw rectangle and label on the original frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the resulting frame
    cv2.imshow('Smile Detection', frame)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()
